You will have to update your notebook's firewall settings to include <tt>*.*.openai.com</tt> in order to get embedddings from OpenAI APIS.

In [19]:
!pip install wget --quiet
!pip install -U openai==0.28 --quiet

In [20]:
import wget

csv_file_path = "https://raw.githubusercontent.com/kubowania/tweet_sentiment_stock_market/main/tweet_sentiment_stock_market.csv"
file_path = "tweet_sentiment_stock_market.csv"

if not os.path.exists(file_path):
    wget.download(csv_file_path ,file_path)
    print("File downloaded successfully.")
else:
    print("File already exists in the local file system.")

File downloaded successfully.


In [21]:
import pandas as pd

df = pd.read_csv(file_path)
df.head()


,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,positive sentiment
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,positive sentiment
2,user I'd be afraid to short AMZN - they are lo...,positive sentiment
3,MNTA Over 12.00,positive sentiment
4,OI Over 21.37,positive sentiment


In [22]:
%%sql
DROP DATABASE IF EXISTS stock_sentiment;
CREATE DATABASE IF NOT EXISTS stock_sentiment;

++
||
++
++

In [23]:
%%sql
USE stock_sentiment;

DROP TABLE IF EXISTS tweets;
CREATE TABLE IF NOT EXISTS tweets (
    text TEXT,
    sentiment TEXT,
    embedding VECTOR(1536) NOT NULL
);

++
||
++
++

In [51]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [25]:
data = df.values.tolist()

In [26]:
import json 

json_data = [json.dumps(row) for row in data]

In [27]:
import openai

openai.api_key = {your_openai_api_key}

embedded_data = []
for text in json_data:
    response = openai.Embedding.create(
        input=[text],
        model="text-embedding-3-small"
    )
    embedding = response["data"][0]["embedding"]
    embedded_data.append(embedding)


In [30]:
combined_data = [tuple(row) + (embedding,) for row, embedding in zip(data, embedded_data)]

In [33]:
s2_df = pd.DataFrame(combined_data)

In [34]:
new_column_names = {0: 'text',
                    1: 'sentiment',
                    2: 'embedding'}

s2_df.rename(columns = new_column_names, inplace = True)

In [35]:
import numpy as np

s2_df['embedding'] = s2_df['embedding'].apply(lambda row: np.array(row).astype(np.float32))

In [36]:
%%sql
USE stock_sentiment;

TRUNCATE TABLE tweets;

++
||
++
++

In [52]:
s2_df.to_sql(
    "tweets",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

999

In [60]:
%%sql

USE stock_sentiment;

SELECT * FROM tweets LIMIT 5;

text,sentiment,embedding
AAP will watch tomorrow for ONG entry.,positive sentiment,[ 3.9982174e-02 -1.6757671e-05 2.9983039e-03 ... 9.9883573e-03 3.8803693e-02 5.7918103e-03]
Won't believe AAP uptrend is back until it crosses above MA(50),negative sentiment,[0.02896267 0.00042855 0.03806976 ... 0.00869674 0.04325869 0.00765102]
SWY - 30% of float short and breaking out - ouch,positive sentiment,[-0.03920249 0.01857298 0.01245161 ... -0.00012916 0.01923171 0.00247626]
BIOF wants 4.90's comin!!!,positive sentiment,[0.00833545 0.00619527 0.01616401 ... 0.03617392 0.02360638 0.01586632]
user BAC For a quick Trade to late..But for investing ~11.98 is a good entry point IMHO,positive sentiment,[-0.00614832 -0.0205384 -0.00455977 ... 0.00697948 0.03302923 0.00815632]
